In [1]:
import pandas as pd
import numpy as np
import scipy as sci

In [2]:
clinical_data = pd.read_csv('F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Datas/Datas/样本临床信息_预处理.csv')
clinical_data

,sample,Sex,Age,Weight,Marital_status,Thyroid_size,Blood_flow,SWE,FT3,FT4,TSH,TPO_Ab,TG_Ab,Group
0,C1,M,33,85.0,Married,0.2440,Normal,23.55,3.36,12.71,0.83,25.52,9.82,C
1,C10,F,55,58.0,Married,0.1728,Normal,13.27,3.27,12.30,3.44,3.20,78.09,C
2,C12,F,33,65.0,Married,0.1924,Normal,13.60,4.14,15.35,3.15,12.85,34.03,C
3,C14,F,35,54.0,Married,0.1557,Normal,9.90,3.14,14.24,0.77,4.02,18.67,C
4,C15,F,33,53.0,Spinster,0.1952,Normal,15.75,3.86,12.74,0.87,5.09,40.46,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,T47,F,59,72.5,Married,0.1591,Rich,41.55,5.36,20.13,4.57,402.93,494.69,T
57,T5,F,53,60.0,Married,0.1401,Slightly rich,55.97,3.23,18.61,4.77,241.74,553.81,T
58,T50,F,57,59.0,Married,0.2904,Rich,43.33,5.64,20.66,7.09,895.97,917.29,T
59,T6,F,29,59.0,Spinster,0.2967,Rich,46.70,3.35,14.31,4.64,745.03,887.38,T


In [43]:
cat_cols = ["Sex","Marital_status"]
num_cols = ["Age","Weight","Thyroid_size"
              ,"SWE","FT3","FT4"
              ,"TSH","TPO_Ab","TG_Ab"]

In [11]:
clinical_data[num_cols+['Group']].groupby('Group').agg([np.mean,np.std])

Age                Weight           Thyroid_size            \
            mean        std       mean       std         mean       std   
Group                                                                     
C      41.800000  13.624015  59.233333  7.600741     0.197983  0.027511   
T      42.129032  12.208582  60.080645  9.649781     0.267768  0.081615   

             SWE                 FT3                  FT4                 TSH  \
            mean       std      mean       std       mean       std      mean   
Group                                                                           
C      14.294000  5.152144  3.702667  0.550560  14.206333  1.729958  1.681000   
T      43.336129  9.994206  4.284194  0.753626  16.581613  2.584214  5.045806   

                     TPO_Ab                   TG_Ab              
            std        mean         std        mean         std  
Group                                                            
C      1.061750   11.910000    7.600729   31.929000   25.357519  
T      0.821522  568.439032  294.900514  697.764194  270.174999

In [35]:
d = {}
d['C']={}
d['T']={}
d['P_value']={}
for i in num_cols:
    data = clinical_data[[i,'Group']]
    T=data[i][data['Group']=='T']
    C=data[i][data['Group']=='C']
    p_value = sci.stats.ttest_ind(T,C)[1]
    stat = data.groupby('Group').agg([np.mean,np.std])
    mean_C = stat.loc['C',(i,'mean')]
    std_C = stat.loc['C',(i,'std')]
    d['C'][i]='{:.1f}'.format(mean_C)+'±'+'{:.1f}'.format(std_C)
    mean_T = stat.loc['T',(i,'mean')]
    std_T = stat.loc['T',(i,'std')]
    # if p_value<0.05:
    #     d['T'][i]='{:.1f}'.format(mean_T)+'±'+'{:.1f}'.format(std_T)+'*'
    # else:
    d['T'][i]='{:.1f}'.format(mean_T)+'±'+'{:.1f}'.format(std_T)
    d['P_value'][i] = '{:.2e}'.format(p_value)
    

In [36]:
d=pd.DataFrame(d)

In [37]:
d

,C,T,P_value
Age,41.8±13.6,42.1±12.2,9.21e-01
Weight,59.2±7.6,60.1±9.6,7.05e-01
Thyroid_size,0.2±0.0,0.3±0.1,3.95e-05
SWE,14.3±5.2,43.3±10.0,1.11e-20
FT3,3.7±0.6,4.3±0.8,1.10e-03
FT4,14.2±1.7,16.6±2.6,9.02e-05
TSH,1.7±1.1,5.0±0.8,3.17e-20
TPO_Ab,11.9±7.6,568.4±294.9,7.50e-15
TG_Ab,31.9±25.4,697.8±270.2,1.32e-19


In [42]:
female_C = sum((clinical_data.Sex=='F')&(clinical_data.Group=='C'))
female_T = sum((clinical_data.Sex=='F')&(clinical_data.Group=='T'))
print(female_T)
print(female_C)

31
28


In [40]:
d.loc['Sex(Female)',]='{}({:.0f}%)'.format(female_C,female_C/30*100),'{}({:.0f}%)'.format(female_T,female_T/31*100)

ValueError: could not broadcast input array from shape (2,) into shape (3,)

In [27]:
d

,C,T
Age/year,41.8±13.6,42.1±12.2
Weight/kg,59.2±7.6,60.1±9.6
Thyroid size/cm³,0.2±0.0,0.3±0.1*
SWE/KPa,14.3±5.2,43.3±10.0*
FT3,3.7±0.6,4.3±0.8*
FT4,14.2±1.7,16.6±2.6*
TSH,1.7±1.1,5.0±0.8*
TPO-Ab,11.9±7.6,568.4±294.9*
TG-Ab,31.9±25.4,697.8±270.2*
Sex(Female),28(93%),31(100%)


In [28]:
married_C = sum((clinical_data['Marital status']=='Married')&(clinical_data.Group=='C'))
married_T = sum((clinical_data['Marital status']=='Married')&(clinical_data.Group=='T'))
d.loc['Marital status(Married)',]='{}({:.0f}%)'.format(married_C,married_C/30*100),'{}({:.0f}%)'.format(married_T,married_T/31*100)

In [29]:
d

,C,T
Age/year,41.8±13.6,42.1±12.2
Weight/kg,59.2±7.6,60.1±9.6
Thyroid size/cm³,0.2±0.0,0.3±0.1*
SWE/KPa,14.3±5.2,43.3±10.0*
FT3,3.7±0.6,4.3±0.8*
FT4,14.2±1.7,16.6±2.6*
TSH,1.7±1.1,5.0±0.8*
TPO-Ab,11.9±7.6,568.4±294.9*
TG-Ab,31.9±25.4,697.8±270.2*
Sex(Female),28(93%),31(100%)


In [30]:
d.index

Index(['Age/year', 'Weight/kg', 'Thyroid size/cm³', 'SWE/KPa', 'FT3', 'FT4',
       'TSH', 'TPO-Ab', 'TG-Ab', 'Sex(Female)', 'Marital status(Married)'],
      dtype='object')

In [31]:
d=d.loc[['Sex(Female)', 'Marital status(Married)','Age/year', 'Weight/kg', 'Thyroid size/cm³', 'SWE/KPa', 'FT3', 'FT4',
       'TSH', 'TPO-Ab', 'TG-Ab'],]

In [32]:
d

,C,T
Sex(Female),28(93%),31(100%)
Marital status(Married),26(87%),25(81%)
Age/year,41.8±13.6,42.1±12.2
Weight/kg,59.2±7.6,60.1±9.6
Thyroid size/cm³,0.2±0.0,0.3±0.1*
SWE/KPa,14.3±5.2,43.3±10.0*
FT3,3.7±0.6,4.3±0.8*
FT4,14.2±1.7,16.6±2.6*
TSH,1.7±1.1,5.0±0.8*
TPO-Ab,11.9±7.6,568.4±294.9*


In [40]:
d.index = ['Sex(Female)', 'Marital status(Married)','Age/year(Mean ± SD)', 'Weight/kg(Mean ± SD)', 'Thyroid size/cm³(Mean ± SD)', 'SWE/KPa(Mean ± SD)', 'FT3(Mean ± SD)', 'FT4(Mean ± SD)',
       'TSH(Mean ± SD)', 'TPO-Ab(Mean ± SD)', 'TG-Ab(Mean ± SD)']

In [41]:
d

,HC,HT
Sex(Female),28(93%),31(100%)
Marital status(Married),26(87%),25(81%)
Age/year(Mean ± SD),41.8±13.6,42.1±12.2
Weight/kg(Mean ± SD),59.2±7.6,60.1±9.6
Thyroid size/cm³(Mean ± SD),0.2±0.0,0.3±0.1*
SWE/KPa(Mean ± SD),14.3±5.2,43.3±10.0*
FT3(Mean ± SD),3.7±0.6,4.3±0.8*
FT4(Mean ± SD),14.2±1.7,16.6±2.6*
TSH(Mean ± SD),1.7±1.1,5.0±0.8*
TPO-Ab(Mean ± SD),11.9±7.6,568.4±294.9*


In [42]:
d.columns = ['HC','HT']

In [43]:
d

,HC,HT
Sex(Female),28(93%),31(100%)
Marital status(Married),26(87%),25(81%)
Age/year(Mean ± SD),41.8±13.6,42.1±12.2
Weight/kg(Mean ± SD),59.2±7.6,60.1±9.6
Thyroid size/cm³(Mean ± SD),0.2±0.0,0.3±0.1*
SWE/KPa(Mean ± SD),14.3±5.2,43.3±10.0*
FT3(Mean ± SD),3.7±0.6,4.3±0.8*
FT4(Mean ± SD),14.2±1.7,16.6±2.6*
TSH(Mean ± SD),1.7±1.1,5.0±0.8*
TPO-Ab(Mean ± SD),11.9±7.6,568.4±294.9*


In [44]:
d.to_csv('F:/my projects/02.论文/论文四_桥本甲状腺炎/Project/Results/clinical features of the study cohort.csv')

In [44]:
clinical_data

,sample,Sex,Age,Weight,Marital_status,Thyroid_size,Blood_flow,SWE,FT3,FT4,TSH,TPO_Ab,TG_Ab,Group
0,C1,M,33,85.0,Married,0.2440,Normal,23.55,3.36,12.71,0.83,25.52,9.82,C
1,C10,F,55,58.0,Married,0.1728,Normal,13.27,3.27,12.30,3.44,3.20,78.09,C
2,C12,F,33,65.0,Married,0.1924,Normal,13.60,4.14,15.35,3.15,12.85,34.03,C
3,C14,F,35,54.0,Married,0.1557,Normal,9.90,3.14,14.24,0.77,4.02,18.67,C
4,C15,F,33,53.0,Spinster,0.1952,Normal,15.75,3.86,12.74,0.87,5.09,40.46,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,T47,F,59,72.5,Married,0.1591,Rich,41.55,5.36,20.13,4.57,402.93,494.69,T
57,T5,F,53,60.0,Married,0.1401,Slightly rich,55.97,3.23,18.61,4.77,241.74,553.81,T
58,T50,F,57,59.0,Married,0.2904,Rich,43.33,5.64,20.66,7.09,895.97,917.29,T
59,T6,F,29,59.0,Spinster,0.2967,Rich,46.70,3.35,14.31,4.64,745.03,887.38,T


In [48]:
clinical_data[cat_cols+['Group']].groupby(['Group','Sex']).agg('count')

Marital_status
Group Sex                
C     F                28
      M                 2
T     F                31

In [50]:
print('{:.2e}'.format(0.7724))

7.72e-01
